In [1]:
import json
from pprint import pprint
import os
import sqlite3
from __future__ import print_function
import unicodedata
import datetime
import numpy as np
from __future__ import division

In [6]:
pathToDB = "/root/jupyter_NB/packetloss-DB-1.db"
pathToNewDB = "/root/jupyter_NB/packetloss-DB-2-downsampled.db"
conn1 = sqlite3.connect(pathToDB)
c1 = conn.cursor()
conn2 = sqlite3.connect(pathToNewDB)
c2 = conn.cursor()

In [7]:
tableCreationString = '''CREATE TABLE rawdata_packetloss (source text, input_source text, 
destination text, tool_name text, input_destination text, 
measurement_agent  text, datapoint_timestamp integer, 
datapoint_packetloss real, topic text, type text, event_timestamp integer, 
summaries text, type_prefix text , event_type text )'''

# check if table already exists if not create it
c2.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='rawdata';")
data =  c2.fetchall()
if data.count(("rawdata_packetloss",)) == 0:
    c2.execute(tableCreationString)
conn2.commit()

In [23]:
def addDataPointToDB(cursor, dataPoint):
    meta = dataPoint["data"]["meta"]
    metadata = dataPoint["metadata"]
    addItems = (meta["source"], meta["input_source"], meta["destination"],
                meta["tool_name"], meta["input_destination"], meta["measurement_agent"], 
                str(dataPoint["data"]["datapoints"]), 
                metadata["topic"], metadata["type"], metadata["kafka_timestamp"], 
                metadata["summaries"], metadata["type_prefix"], metadata["event-type"])
    cursor.execute("INSERT INTO rawdata VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)", addItems)


# returns events, that went from source to dest as a list
# the list is ordered in time, where the newest event comes first
def getDataPoints(source, dest, coursor):
    dataTuples = []
    query_info = (source, dest, )
    counter = 0
    for row in coursor.execute('SELECT * FROM rawdata WHERE input_source=? AND input_destination=? ORDER BY event_timestamp DESC', query_info):
        counter += 1
        dataTuples.append(row)
    print(str(counter) + " Datapoints were read.")
    # store all datapoints in a fitting format
    dataPoints = []
    for point in dataTuples:
        # do unicode converstion and formatting
        text = point[6].encode('ascii','ignore').replace("u", '').replace("'", '"')
        data = json.loads(text)
        dataPoints.append(data)
    return (dataTuples, dataPoints)



In [24]:
# get all source/dest pairs
source = "perfsonar-lt.cern.ch"
dest = "ps-latency.lhcmon.triumf.ca"
testDataPoints = []
testDataPoints = getDataPoints(source, dest, c)

1018 Datapoints were read.


In [27]:
pprint(testDataPoints[0][0])
pprint(testDataPoints[1][0])

(u'128.142.223.247',
 u'perfsonar-lt.cern.ch',
 u'206.12.9.2',
 u'powstream',
 u'ps-latency.lhcmon.triumf.ca',
 u'128.142.223.247',
 u"{u'1471868355': 0.0, u'1471870683': 0.0, u'1471868115': 0.0, u'1471868295': 0.0, u'1471870624': 0.0016666666666666668, u'1471868662': 0.0, u'1471868421': 0.0, u'1471868543': 0.0, u'1471868237': 0.0, u'1471869377': 0.0, u'1471869846': 0.0, u'1471869199': 0.0, u'1471870504': 0.0, u'1471869967': 0.0, u'1471869676': 0.0, u'1471868962': 0.0, u'1471869789': 0.0, u'1471869908': 0.0, u'1471870325': 0.0, u'1471868177': 0.0, u'1471868845': 0.0, u'1471870087': 0.0, u'1471870444': 0.0, u'1471869500': 0.0, u'1471870146': 0.0, u'1471870203': 0.0, u'1471869438': 0.0, u'1471869022': 0.0, u'1471869140': 0.0, u'1471868906': 0.0, u'1471870029': 0.0, u'1471868783': 0.0, u'1471870385': 0.0, u'1471870267': 0.07333333333333333, u'1471869561': 0.0, u'1471869732': 0.0, u'1471868604': 0.0, u'1471868481': 0.0, u'1471869320': 0.0, u'1471869619': 0.0, u'1471870564': 0.0, u'14718680